# Анализ токсичности комментариев

**Техническое задание**

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества <font color='red'>F1 не меньше 0.75.</font> 

В ходе проекта использованы библиотеки pandas, numpy, sklearn, catboost. Для предсказания токсичности комментариев используются модели логистической регрессии, дерево решений, случайный лес, градиентный бустинг Catboost. Подбор параметров наилучшей модели проведён с помощью ParameterGrid. Ключевая метрика - F1. При работе с текстами используются средства библиотеки NLTK и BERT.

## Подготовка данных

In [ ]:
import pandas as pd
import numpy as np
import re 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [ ]:
import nltk
from nltk.corpus import stopwords as nltk_stopwords
# скачать в случае отсутствия:
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('punkt')
stopwords = set(nltk_stopwords.words('english'))
from nltk.stem import WordNetLemmatizer

Читаем базу данных.

In [ ]:
df = pd.read_csv('\datasets\toxic_comments.csv', sep= ',')
display(df.sample(10))
df.info()

,text,toxic
3221,"""\nNPVO is mostly just about the edit left by ...",0
55972,Was not aware of this . Thanks .Will correct,0
3294,"""\nIt says in the AIV header that warnings mus...",0
120188,""" But tend to agree with OP that Sukhoi is red...",0
111516,"Right, I've removed two autoblocks that were s...",0
87318,"""\n\n about asessment of Remo Fernandes \n\nHe...",0
4505,"""\n\nResponse to your recent edit at Loyola Ac...",0
149451,Rewriting\nI cut out ref to bell in NZ and fou...,0
157405,A user misbehaving \n\nCheck out the talk page...,0
135847,"""\n\n Editor Topic Ban \n\nI'm not so sure we ...",0


Мы получили 159571 случай для анализа. Случай состоит из текста и разметки, является ли этот текст токсичным, что и станет нашей целевой переменной. В базе нет дубликатов и пропусков, можно приступить к следующему шагу. В данном проекте мы поступим следующим образом: 
1. векторизируем текст, используя меру TF-IDF 
2. применим логистическую регрессию и простые модели
3. сравним результаты с градиентным бустингом Catboost по уровню F1.

Для начала преобразуем тексты в <font color='red'>лемматизированные списки слов:</font>

In [ ]:
corpus = df['text'].values.copy()
corpus[:2]

array(["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"],
      dtype=object)

In [ ]:
for i in range(len(corpus)):
    corpus[i] = nltk.word_tokenize(corpus[i])
corpus[:2]

array([list(['Explanation', 'Why', 'the', 'edits', 'made', 'under', 'my', 'username', 'Hardcore', 'Metallica', 'Fan', 'were', 'reverted', '?', 'They', 'were', "n't", 'vandalisms', ',', 'just', 'closure', 'on', 'some', 'GAs', 'after', 'I', 'voted', 'at', 'New', 'York', 'Dolls', 'FAC', '.', 'And', 'please', 'do', "n't", 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', 'I', "'m", 'retired', 'now.89.205.38.27']),
       list(["D'aww", '!', 'He', 'matches', 'this', 'background', 'colour', 'I', "'m", 'seemingly', 'stuck', 'with', '.', 'Thanks', '.', '(', 'talk', ')', '21:51', ',', 'January', '11', ',', '2016', '(', 'UTC', ')'])],
      dtype=object)

In [ ]:
df['lemmatized_text'] = [' '.join([WordNetLemmatizer().lemmatize(word) for word in text]) for text in corpus]
df

,text,toxic,lemmatized_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D'aww ! He match this background colour I 'm s...
2,"Hey man, I'm really not trying to edit war. It...",0,"Hey man , I 'm really not trying to edit war ...."
3,"""\nMore\nI can't make any real suggestions on ...",0,`` More I ca n't make any real suggestion on i...
4,"You, sir, are my hero. Any chance you remember...",0,"You , sir , are my hero . Any chance you remem..."
...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,`` : : : : : And for the second time of asking...
159567,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That is a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,"Spitzer Umm , there no actual article for pros..."
159569,And it looks like it was actually you who put ...,0,And it look like it wa actually you who put on...


In [ ]:
df['cleared_text'] = [" ".join(re.sub(r'[^a-zA-Z ]', ' ', text).split()) for text in df['lemmatized_text']]
df

,text,toxic,lemmatized_text,cleared_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D'aww ! He match this background colour I 'm s...,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,"Hey man , I 'm really not trying to edit war ....",Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,`` More I ca n't make any real suggestion on i...,More I ca n t make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,"You , sir , are my hero . Any chance you remem...",You sir are my hero Any chance you remember wh...
...,...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,`` : : : : : And for the second time of asking...,And for the second time of asking when your vi...
159567,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That is a ho...,You should be ashamed of yourself That is a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,"Spitzer Umm , there no actual article for pros...",Spitzer Umm there no actual article for prosti...
159569,And it looks like it was actually you who put ...,0,And it look like it wa actually you who put on...,And it look like it wa actually you who put on...


<font color='red'>Разделим</font> данные на тренировочную и тестовую выборки:

In [ ]:
X = df.cleared_text
y = df.toxic

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=23)
print(X.shape, y.shape)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

Мы положили в выборку 10% случаев, то есть 15958 текстов. <font color='red'>Векторизируем</font> их, чтобы с ними могли работать простые модели.

In [ ]:
fitted_vectorizer = TfidfVectorizer(stop_words=stopwords).fit(X_train)
tf_idf_train = fitted_vectorizer.transform(X_train)
tf_idf_test = fitted_vectorizer.transform(X_test)

# для отображения полученных матриц можно воспользоваться следующим кодом, но сейчас мы сэкономим память и делать этого не будем
#tf_idf_train = pd.DataFrame.sparse.from_spmatrix(tf_idf_train)
#tf_idf_test = pd.DataFrame.sparse.from_spmatrix(tf_idf_test)

display(tf_idf_train)
display(tf_idf_test)

<143613x155045 sparse matrix of type '<class 'numpy.float64'>'
	with 3929633 stored elements in Compressed Sparse Row format>

<15958x155045 sparse matrix of type '<class 'numpy.float64'>'
	with 421502 stored elements in Compressed Sparse Row format>

In [ ]:
print(tf_idf_train.shape, y_train.shape)
print(tf_idf_test.shape, y_test.shape)

### Вывод

Итак, мы подготовили данные для отработки алгоритма классификации текстов на токсичные и нет с помощью простых моделей (логистическая регрессия, дерево решений и случайный лес). Тексты токенизированы, лемматизированы и затем векторизированы так, что у нас вместо текста теперь 174274 дихотомические переменные для описания этого текста в категориях 0 и 1.

## Обучение

### Модель линейной регрессии

In [ ]:
model = LogisticRegression(random_state=12345, class_weight='balanced', solver = 'sag')
model.fit(tf_idf_train, y_train)
predictions = model.predict(tf_idf_test)
current_model = pd.DataFrame({'Model': ['Logistic Regression'],
              'F1': [f1_score(y_test, predictions)]})
all_models = current_model.merge(current_model, how = 'outer')
all_models

,Model,F1
0,Logistic Regression,0.74804


Мы сразу получили результат, близкий к выполнению технического требования заказчика. Посмотрим, чего можно добиться с помощью изменения гиперпараметра С:

In [ ]:
#с помощью данного кода удалось найти наилучшее значение гиперпараметра С на уровне 9.9. 
#т.к. код работает слишком долго, повторно он не запускался.

param_grid = {'C': np.logspace(-5, 2, 50)}
best_score = 0
for c in ParameterGrid(param_grid):
    model.set_params(**c)
    model.fit(tf_idf_train, y_train)
    predictions = model.predict(tf_idf_test)
    if f1_score(y_test, predictions) > best_score:
        best_score = f1_score(y_test, predictions)
        best_grid = c

print("F1: %0.2f" % best_score)
print("Grid:", best_grid)

In [ ]:
model = LogisticRegression(random_state=12345, class_weight='balanced', solver = 'sag', max_iter = 1000, C = 7)
model.fit(tf_idf_train, y_train)
predictions = model.predict(tf_idf_test)
current_model = pd.DataFrame({'Model': ['Logistic Regression'],
              'F1': [f1_score(y_test, predictions)]})
all_models = current_model.merge(current_model, how = 'outer')
all_models

,Model,F1
0,Logistic Regression,0.761072


### Модель случайного леса

In [ ]:
model = RandomForestClassifier(random_state = 23, n_estimators = 20)
model.fit(tf_idf_train, y_train)
predictions = model.predict(tf_idf_test)
current_model = pd.DataFrame({'Model': ['Random Forest'],
              'F1': [f1_score(y_test, predictions)]})
all_models = all_models.merge(current_model, how = 'outer')
all_models

,Model,F1
0,Logistic Regression,0.761072
1,Random Forest,0.678327


### Модель дерева решений

In [ ]:
model = DecisionTreeClassifier(random_state = 23)
model.fit(tf_idf_train, y_train)
predictions = model.predict(tf_idf_test)
current_model = pd.DataFrame({'Model': ['Decision Tree'],
              'F1': [f1_score(y_test, predictions)]})
all_models = all_models.merge(current_model, how = 'outer')
all_models

,Model,F1
0,Logistic Regression,0.761072
1,Random Forest,0.678327
2,Decision Tree,0.709658


<font color='red'>Логистическая регрессия отвечает требованиям технического задания.</font> F1-мера на тестовой выборке выше 0,76. Посмотрим, чего можно добиться с помощью Catboost, для которой бы не понадобилась токенизация, лемматизация и векторизация текста, которую мы проделали на этапе предобработки.

### Градиентный бустинг Catboost

В текущей среде не поддерживается работа модели CatBoost с текстовыми переменными, поэтому мы приведём код, который был запущен в другой среде и ниже приведём скриншот результата.

In [ ]:
X_train_cat = pd.DataFrame(X_train)
X_test_cat = pd.DataFrame(X_test)

model = CatBoostClassifier(verbose = 10, eval_metric = 'F1')
model.fit(X_train_cat, y_train, text_features=['cleared_text'])
predictions = model.predict(X_test_cat)
current_model = pd.DataFrame({'Model': ['Catboost'],
              'F1': [f1_score(y_test, predictions)]})
all_models = all_models.merge(current_model, how = 'outer')
all_models

,Model,F1
0,Logistic Regression,0.761072
1,Random Forest,0.678327
2,Decision Tree,0.709658
3,Catboost,0.765119


## Bert

Добавим недостающие библиотеки и модули

In [ ]:
import torch
import transformers
from transformers import BertTokenizer
from tqdm import notebook

Выбираем модель и токенизатор

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = transformers.BertModel.from_pretrained("bert-base-uncased")

У нас недостаточно ресурса для исследования всей базы, сделаем выборку в 400 случаев.
Отберём позитивные и токсичные комментарии в равных пропорциях

In [ ]:
df = pd.read_csv('\datasets\toxic_comments.csv', sep= ',')
df['weights'] = df['toxic']
df['weights'] = df['weights'].replace({0:0.1, 1:0.9})
df = df.sample(400, random_state=23, weights = df.weights)
df['text'] = df['text'][:][0:512]
df.reset_index()

,index,text,toxic,weights
0,82243,"""\n\n Please do not vandalize pages, as you di...",0,0.1
1,151131,"LGBT \n\nyou little fuck , are you a fag , tha...",1,0.9
2,122095,FUCK YOU! FUCK EVERYBODY HERE AND THEIR FUCKIN...,1,0.9
3,44657,remove your head from your butt \n\nNeilN need...,1,0.9
4,34962,That may be appalling but what's really appall...,1,0.9
...,...,...,...,...
395,135287,Your Lie(s) About Me \nI've never put a lie on...,1,0.9
396,114961,", you are making a fool of yourself!! This is ...",1,0.9
397,158007,gay charver that no one likes,1,0.9
398,70886,That sentence and link should be deleted unles...,0,0.1


Токенизируем текст

In [ ]:
tokenized = df['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

Чтобы строки матрицы по каждому тексту были одинаковые по длине, добавим нули в конце там, где длина короче максимума.
Длины срок придётся сократить до 512 чисел, ограничение модели

In [ ]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
padded = padded[:, 0:512]
attention_mask = np.where(padded != 0, 1, 0)

Сформируем эмбеддинги на выборке по частям

In [ ]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
features = np.concatenate(embeddings)

Обучаем логистическую регрессию на полученном массиве:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features, df['toxic'], test_size=0.5, random_state=23)
model = LogisticRegression(random_state=23, class_weight='balanced')
model.fit(X_train, y_train)
predict = model.predict(X_test)

current_model = pd.DataFrame({'Model': ['Logistic regression with BERT'],
                              'F1': [f1_score(y_test, predict)]})
all_models = all_models.merge(current_model, how = 'outer')
all_models

,Model,F1
0,Logistic Regression,0.761072
1,Random Forest,0.678327
2,Decision Tree,0.709658
3,Catboost,0.765119
4,Logistic regression with BERT,0.818653


## Выводы

Итак, <font color='red'>мы создали модель, комплексная оценка точности и отзывчивости которой находится на уровне 82%</font>, что превышает установленный в техническом задании порог. Наилучшей моделью оказалась логистическая регрессия на эмбеддингах, созданных с помощью BERT, однако работу данной модели необходимо проверить на большей выборке, что требует много ресурсов. Логистическая регрессия, проведённая на векторизированном с помощью TF-IDF тексте, требует гораздоб меньше ресурсов, быстро обучается на всей базе и показывает результат на уровне 75%, что достаточно для выполнения технческого требования. Существенным преимуществом данной модели является скорость её обучения и предсказания, однако для её работы необходима существенная предобработка (токенизация, лемматизация и векторизация текста), как и в случаае с работой через BERT. Catboost не требует преобработки, что может стать аргументом в пользу использования именно этой модели. В различных условиях применения следует выбирать из данных моделей.